In [ ]:
!pip install lxml

In [287]:
import requests
from lxml import etree, html
from itertools import chain
import pandas as pd
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

resp = requests.get("http://www.nemoapps.com/phrasebooks/hebrew")
base = "http://www.nemoapps.com"

htmlparser = etree.HTMLParser()
tree = etree.fromstring(resp.text, htmlparser)



In [288]:
string_document = html.fromstring(resp.text)
links = list(set(string_document.iterlinks()))
audio = []

for link in links:
    if link[2].endswith(".mp3"):
        ln = link[2]
        #if not ln.startswith(base):
        #    ln = base + ln
        audio.append(ln)
print ("Length of the link : " + str(len(audio)))

Length of the link : 101


In [289]:
def stringify_children(node):
    parts = (list(chain(*(etree.tostring(c, encoding="utf-8").decode().split() for c in node.getchildren()))))
    return ' '.join(filter(None, parts))

In [290]:
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext.strip()

In [291]:
def _walk_parents(tag):
    if tag.tag in ["source", "audio"]:
        tag = tag.getparent()
        return _walk_parents(tag)
    else:
        return(tag)


In [292]:
def _detect_tag(link: str, tag: str, attr: str):
    A = tree.xpath(f"//{tag}[@{attr}='{link}']")
    if len(A) > 0:
        return A[0]
    else:
        return None

In [293]:
#first tag containing audio link
L = None
elem = None
atrib = None
#tags to try
tags = [("source", "src"),("audio", "src"),("div", "downloadURL"),("div", "src")]
for tg in tags:
    A = _detect_tag(audio[0], tg[0], tg[1])
    if A is not None:
        L = _walk_parents(A)
        elem = tg[0]
        atrib = tg[1]
        break
    else:
        print (f"Exception: cannot extract {tg[0]} tag")

print(L.tag)
if L.tag == "td":
    L = L.getparent() # the looping element must be the tr
#get looping siblings
loops = [L]
loops += list(L.itersiblings())
loops += list(L.itersiblings(preceding=True))
print(len(loops))

td
101


In [294]:
#iterate loops. extract audio url, text and attributes
df = pd.DataFrame(columns=["url","label"])
for loop in loops:
    items = list(loop.iter())
    ln = None
    for item in items:
        if item.get(atrib) is not None:
            if item.get(atrib).endswith(".mp3"):
                ln = item.get(atrib)
                if not ln.startswith(base):
                    ln = base + ln
            break
    label = cleanhtml(stringify_children(loop))
    df = df.append({'url':ln, 'label':label}, ignore_index = True)

print(df.shape)

df.to_parquet("demo_audio.parquet")

print(df.head())

(101, 2)
